In [1]:
import torch
from PIL import ImageFont, ImageDraw, Image
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import easyocr
import cv2
import matplotlib.pyplot as plt
import math
from sklearn.metrics import r2_score
import time

CKPT_PATH = './best.pt'
yolov5 = torch.hub.load('ultralytics/yolov5',
                        'custom',
                        path=CKPT_PATH)

'confidence: ' + str(yolov5.conf)
reader = easyocr.Reader(['ko'])

/Users/idahyeon/q/envs/newenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in /Users/idahyeon/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-10-8 Python-3.8.13 torch-1.12.1 CPU

Fusing layers... 
Model summary: 378 layers, 35248920 parameters, 0 gradients
Adding AutoShape... 
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


# 1차 상헌 상수 - RSquared

In [2]:
total_result=[]
total=[]
result_720=[]
result_1080=[]
result_2160=[]

vedios = os.listdir('./total_vedio/720p/')
vedios.sort()

for pth in vedios:
    print("==================")
    print("pth : " + str(pth))
    
    
    cap = cv2.VideoCapture("./total_vedio/720p/"+str(pth))
    
    
    width = cv2.CAP_PROP_FRAME_WIDTH
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    
    
    if "100" in pth : m=1
    elif "150" in pth : m=1.5
    elif "200" in pth : m=2
    elif "250" in pth : m=2.5
    elif "300" in pth : m=3
    elif "350" in pth : m=3.5
    elif "400" in pth : m=4
    elif "450" in pth : m=4.5
    elif "500" in pth : m=5
    elif "550" in pth : m=5.5
    elif "600" in pth : m=6
    elif "650" in pth : m=6.5
    elif "700" in pth : m=7
    
    if pth==".DS_Store": continue
    while cv2.waitKey(1) <0:
        hasFrame, frame = cap.read()
        pred_df = pd.DataFrame()
        
        if not hasFrame :   #프레임 끝났을 경우 종료 
            # print("Done Processing !!")
            cv2.waitKey(3000)
            break
        
        pred = yolov5(frame, size=1280, augment=False) # 번호판 인식  
        
        for i, row in pred.pandas().xyxy[0].iterrows(): #객체 정확도 분별
            if row['confidence'] < yolov5.conf: break
            pred_df = pred_df.append(row)
            
        pred_df = pred_df.reset_index()
        pred.save()
        
        for i, row in pred_df.iterrows(): # 한 프레임에서 인식한 객체 
            #거리구하기
            area = (row['ymax']-row['ymin']-8)*(row['xmax']-row['xmin']-8)  #넓이
            
            if(float(720)==height):
                distance = round(135/math.sqrt(area),2)
                print("상수 : 135")
            else: distance=0
            
            total_result.append(distance)
            total.append(m)
            
            print("예측 거리 : "+ str(distance))
            print("실제 거리 : "+ str(m))
            
    cap.release()

# total = [1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7]

# total_result = one_result+three_result+five_result+seven_result

print("rs 라이브러리 -1차 상헌상수 : "+ str(r2_score(total, total_result)))

from sklearn.metrics import mean_absolute_error
print("MAE - 1차 상헌 상수 : "+str(mean_absolute_error(total, total_result)))

from sklearn.metrics import mean_squared_error 
MSE_con=mean_squared_error(total, total_result)
print("MSE - 1차 상헌 상수 : "+str(mean_squared_error(total, total_result)))

from sklearn.metrics import mean_squared_error 
print("RMSE - 1차 상헌 상수: "+str(np.sqrt(MSE_con)))


pth : .DS_Store
pth : 100cm_720p.MOV


OpenCV: Couldn't read video stream from file "./total_vedio/720p/.DS_Store"
[ERROR:0] global /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-8m8bbyhv/opencv/modules/videoio/src/cap.cpp (166) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.5.4) /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-8m8bbyhv/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): ./total_vedio/720p/.DS_Store in function 'icvExtractPattern'


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_6195/560341000.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998


상수 : 135
예측 거리 : 1.05
실제 거리 : 1


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_6195/560341000.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998


상수 : 135
예측 거리 : 1.05
실제 거리 : 1


...

상수 : 135
예측 거리 : 9.26
실제 거리 : 7


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_6195/560341000.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998


상수 : 135
예측 거리 : 9.2
실제 거리 : 7


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_6195/560341000.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998


상수 : 135
예측 거리 : 9.21
실제 거리 : 7
rs 라이브러리 -1차 상헌상수 : 0.7723081513652632
MAE - 1차 상헌 상수 : 0.5558943577430973
MSE - 1차 상헌 상수 : 0.7996629051620646
RMSE - 1차 상헌 상수: 0.8942387294017546


# 5m 가시화

In [9]:
##가로, 세로 길이 기반 측정
def grd_length(resolution, width, height):
    #normalize value
    mean = 520.2006621958521
    std = 675.1471686772571

    #normalizing
    resolution = (resolution-mean) / std
    height = (height-mean) / std
    width = (width-mean) / std
    
    #calculating distance
    distance = (-32.62872226) + (1.36456181*resolution) + (3.83042209*width) + ((-52.93560358)*height)
    
    return distance



##넓이 기반 측정
def grd_size(resolution, size):
    #normalize value
    mean = 708.7067193723738
    std = 750.7078400469876

    #normalizing
    resolution = (resolution-mean) / std
    size = (size-mean) / std
    
    #calculating distance
    distance = (-12.97574844) + (1.36510736*resolution) + ((-19.48221397)*size)
    
    return distance

def grd_length_resolution(resolution, width, height):
    distance = 0
    
    if resolution == 720:
        #normalize value
        mean = 67.82630844999002
        std = 61.346605511051685
        
        #normalizing
        width = (width-mean) / std
        height = (height-mean) / std
        
        #calculating distance
        distance = (-18.58963292) + (5.40696279*width) + ((-29.74396128)*height)
    
    elif resolution == 1080:
        #normalize value
        mean = 100.09815308569979
        std = 90.0449028230475
        
        #normalizing
        width = (width-mean) / std
        height = (height-mean) / std
        
        #calculating distance
        distance = (-5.48210169) + (1.38873641*width) + ((-13.10648836)*height)
        
    elif resolution == 2160:
        #normalize value
        mean = 192.97851834564483
        std = 173.24456702735597
        
        #normalizing
        width = (width-mean) / std
        height = (height-mean) / std
        
        #calculating distance
        distance = (-2.01397684) + (0.425009*width) + ((-8.78440094)*height)
        
    
    
    return distance
    

def grd_size_resolution(resolution, size):
    distance = 0
    
    if resolution == 720:
        #normalize value
        mean = 55.162371024747586
        std = 31.419484714306954
        
        #normalizing
        size = (size-mean) / std
        
        #calculating distance
        distance = (4) + ((-1.68509687)*size)
    
    elif resolution == 1080:
        #normalize value
        mean = 80.87600296910803
        std = 44.903575372439505
        
        #normalizing
        size = (size-mean) / std
        
        #calculating distance
        distance = (4) + ((-1.67382192)*size)
        
    elif resolution == 2160:
        #normalize value
        mean = 156.2019422403862
        std = 87.05443271571005
        
        #normalizing
        size = (size-mean) / std
        
        #calculating distance
        distance = (4) + ((-1.68307815)*size)
        
    
    
    return distance

cap = cv2.VideoCapture("./total_vedio/data/21.MOV")
outputFile = "result_3차상헌상수.mp4"
font=ImageFont.truetype(os.path.join("./font/Nanum_Gothic/NanumGothic-Bold.ttf"),50)
vid_writer = cv2.VideoWriter(outputFile, cv2.VideoWriter_fourcc('D', 'I', 'V', 'X'), cap.get(cv2.CAP_PROP_FPS), (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)


reader = easyocr.Reader(['ko'])
while cv2.waitKey(1) <0:
    result =[]
    hasFrame, frame = cap.read()
    pred_df = pd.DataFrame()
    if not hasFrame :
        print("Done Processing !!")
        cv2.waitKey(3000)
        break
    pred = yolov5(frame, size=1280, augment=False) # 번호판 인식  
    for i, row in pred.pandas().xyxy[0].iterrows():
        if row['confidence'] < yolov5.conf: break
        pred_df = pred_df.append(row)
    pred_df = pred_df.reset_index()
    pred.save()
    cv2.imwrite("frame.jpg",frame)
    img_pil = Image.fromarray(frame)
    draw = ImageDraw.Draw(img_pil)
    for i, row in pred_df.iterrows():
        img = np.asarray(frame)
        cnt=0
        imageText=img.copy()
        img_cropped = img[int(row['ymin']):int(row['ymax']), int(row['xmin']):int(row['xmax'])]
        img_gray = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2GRAY)
        plt.subplot(4,4,i+1)
        
        #거리구하기
        # area = (row['ymax']-row['ymin']-8)*(row['xmax']-row['xmin']-8)
        area = (row['ymax']-row['ymin'])*(row['xmax']-row['xmin'])
        # distance = round(135/math.sqrt(area),2) #1차 상헌상수
        # distance = round(167/math.sqrt(area),2) #2차 상헌상수
        # distance =round(grd_length(float(height),(row['xmax']-row['xmin']),(row['ymax']-row['ymin'])),2) #grd_length
        # distance = round(grd_size(float(height),math.sqrt(area)),2)  #grd_size
        # distance=round(grd_length_resolution(float(height),row['xmax']-row['xmin'],row['ymax']-row['ymin']),2) #grd_length_resolution
        #distance = round(grd_size_resolution(float(height),math.sqrt(area)),2) #grd_size_resolution
        
        
        con=0.215972*height + 11.5
        distance = round(con/math.sqrt(area),2)
            
        print(distance)
        
        draw.text((int(row['xmin'])+i*150,int(row['ymax'])+10),(str(distance)+" m"),font=font,fill=(0,0,255))
        
        break
        
        
    img = np.array(img_pil)
    vid_writer.write(img)
    print(result)


vid_writer.release()
cap.release()

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_96632/1050161838.py:137: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998


4.86
[]


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_96632/1050161838.py:137: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998


4.87
[]


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_96632/1050161838.py:137: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998


...

5.08
[]


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_96632/1050161838.py:137: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998


5.11
[]


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_96632/1050161838.py:137: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998


5.08
[]
Done Processing !!


#  정확도 구하기 - 화질에 따른 거리와 번호판의 넓이의 제곱근의 관계 유도식

In [6]:
total_result=[]
total=[]

# total_result_720=[]  #새로운 상수 - 예측값
# total_result_1080=[] #새로운 상수 - 예측값
# total_result_2160=[] #새로운 상수 - 예측값
# total_720=[] #새로운 상수 - 관측값
# total_1080=[] #새로운 상수 - 관측값
# total_2160=[] #새로운 상수 - 관측값

total_result_grd_length=[]
total_result_grd_size=[]
total_result_grd_length_resolution=[]
total_result_grd_size_resolution=[]
total_result_new_con=[]


vedios = os.listdir('./total_vedio/pixel_vedio/')
vedios.sort()

for pth in vedios:
    print("==================")
    print("pth : " + str(pth))
    
    
    cap = cv2.VideoCapture("./total_vedio/pixel_vedio/"+str(pth))
    
    
    width = cv2.CAP_PROP_FRAME_WIDTH
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    
    
    if "100" in pth : m=1
    elif "150" in pth : m=1.5
    elif "200" in pth : m=2
    elif "250" in pth : m=2.5
    elif "300" in pth : m=3
    elif "350" in pth : m=3.5
    elif "400" in pth : m=4
    elif "450" in pth : m=4.5
    elif "500" in pth : m=5
    elif "550" in pth : m=5.5
    elif "600" in pth : m=6
    elif "650" in pth : m=6.5
    elif "700" in pth : m=7
    
    if pth==".DS_Store": continue
    while cv2.waitKey(1) <0:
        hasFrame, frame = cap.read()
        pred_df = pd.DataFrame()
        
        if not hasFrame :   #프레임 끝났을 경우 종료 
            # print("Done Processing !!")
            cv2.waitKey(3000)
            break
        
        pred = yolov5(frame, size=1280, augment=False) # 번호판 인식  
        
        for i, row in pred.pandas().xyxy[0].iterrows(): #객체 정확도 분별
            if row['confidence'] < yolov5.conf: break
            pred_df = pred_df.append(row)
            
        pred_df = pred_df.reset_index()
        pred.save()
        
        for i, row in pred_df.iterrows(): # 한 프레임에서 인식한 객체 
            #거리구하기
            # area = (row['ymax']-row['ymin']-8)*(row['xmax']-row['xmin']-8)  #넓이
            area = (row['ymax']-row['ymin'])*(row['xmax']-row['xmin'])  #넓이
            
            
            # total_result_grd_length.append(grd_length(float(height),row['xmax']-row['xmin'],row['ymax']-row['ymin']))
            # total_result_grd_size.append(grd_size(float(height),math.sqrt(area)))
            # total_result_grd_length_resolution.append(grd_length_resolution(float(height),row['xmax']-row['xmin'],row['ymax']-row['ymin']))
            # total_result_grd_size_resolution.append(grd_size_resolution(float(height),math.sqrt(area)))
            
            # if(float(720)==height):
            #     distance = round(167/math.sqrt(area),2)
            #     print("상수 : 167")
            # elif(float(1080)==height):
            #     distance = round(248/math.sqrt(area),2)
            #     print("상수 : 248")
            # elif(float(2160)==height):
            #     distance = round(478/math.sqrt(area),2)
            #     print("상수 : 478")
            # else: distance=0
            
            # total_result_new_con.append(distance)
            
            
            con=0.215972*height + 11.5
            distance = round(con/math.sqrt(area),2)
            total_result.append(distance)
            
            total.append(m)
            
            print("실제 거리 : "+ str(m))
            # print("grd_length : "+str(grd_length(float(height),row['xmax']-row['xmin'],row['ymax']-row['ymin'])))
            # print("grd_size : "+str(grd_size(float(height),math.sqrt(area))))
            # print("grd_length_resolution : "+str(grd_length_resolution(float(height),row['xmax']-row['xmin'],row['ymax']-row['ymin'])))
            # print("grd_size_resolution : "+str(grd_size_resolution(float(height),math.sqrt(area))))
            # print("상헌 상수 예측 거리 : "+ str(distance))
            print("예측 거리 : "+ str(distance))
            
            
    cap.release()

# print("rs 라이브러리 _ grd_length : "+ str(r2_score(total, total_result_grd_length)))
# print("rs 라이브러리 _ grd_size : "+ str(r2_score(total, total_result_grd_size)))
# print("rs 라이브러리 _ grd_length_resolution : "+ str(r2_score(total, total_result_grd_length_resolution)))
# print("rs 라이브러리 _ grd_size_resolution : "+ str(r2_score(total, total_result_grd_size_resolution)))
# print("rs 라이브러리 _ 새로운 상헌 상수 : "+ str(r2_score(total, total_result_new_con)))


from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(total,)
# MAE_grd_length=mean_absolute_error(total, total_result_grd_length)
# MAE_grd_size=mean_absolute_error(total, total_result_grd_size)
# MAE_grd_length_resolution=mean_absolute_error(total, total_result_grd_length_resolution)
# MAE_grd_size_resolution=mean_absolute_error(total, total_result_grd_size_resolution)
# MAE_new_con=mean_absolute_error(total, total_result_new_con)
print("MAE : "+str(mean_absolute_error(total, )))
# print("MAE - grd_length: "+str(mean_absolute_error(total, total_result_grd_length)))
# print("MAE - grd_size : "+str(mean_absolute_error(total, total_result_grd_size)))
# print("MAE - grd_length_resolution : "+str(mean_absolute_error(total, total_result_grd_length_resolution)))
# print("MAE - grd_size_resolution : "+str(mean_absolute_error(total, total_result_grd_size_resolution)))
# print("MAE - 새로운 상헌 상수 : "+str(mean_absolute_error(total, total_result_new_con)))

from sklearn.metrics import mean_squared_error 
MSE = mean_squared_error(total,)
# MSE_grd_length=mean_squared_error(total, total_result_grd_length)
# MSE_grd_size=mean_squared_error(total, total_result_grd_size)
# MSE_grd_length_resolution=mean_squared_error(total, total_result_grd_length_resolution)
# MSE_grd_size_resolution=mean_squared_error(total, total_result_grd_size_resolution)
# MSE_new_con=mean_squared_error(total, total_result_new_con)
print("MSE : "+str(mean_squared_error(total, )))
# print("MSE - grd_length: "+str(mean_squared_error(total, total_result_grd_length)))
# print("MSE - grd_size : "+str(mean_squared_error(total, total_result_grd_size)))
# print("MSE - grd_length_resolution : "+str(mean_squared_error(total, total_result_grd_length_resolution)))
# print("MSE - grd_size_resolution : "+str(mean_squared_error(total, total_result_grd_size_resolution)))
# print("MSE - 새로운 상헌 상수 : "+str(mean_squared_error(total, total_result_new_con)))

from sklearn.metrics import mean_squared_error 
print("RMSE - grd_length: "+str(np.sqrt(MSE)))
# print("RMSE - grd_length: "+str(np.sqrt(MSE_grd_length)))
# print("RMSE - grd_size: "+str(np.sqrt(MSE_grd_size)))
# print("RMSE - grd_length_resolution: "+str(np.sqrt(MSE_grd_length_resolution)))
# print("RMSE - grd_size_resolution: "+str(np.sqrt(MSE_grd_size_resolution)))
# print("RMSE - 새로운 상헌 상수: "+str(np.sqrt(MSE_new_con)))

pth : .DS_Store
pth : 100_1080p.MOV


OpenCV: Couldn't read video stream from file "./total_vedio/pixel_vedio/.DS_Store"
[ERROR:0] global /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-8m8bbyhv/opencv/modules/videoio/src/cap.cpp (166) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.5.4) /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-8m8bbyhv/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): ./total_vedio/pixel_vedio/.DS_Store in function 'icvExtractPattern'


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_73700/454715945.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9990


실제 거리 : 1
예측 거리 : 1.23


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_73700/454715945.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9991


실제 거리 : 1
예측 거리 : 1.23


...

/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_73700/454715945.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998


실제 거리 : 7
예측 거리 : 6.63


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_73700/454715945.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998


실제 거리 : 7
예측 거리 : 6.66


ValueError: Found input variables with inconsistent numbers of samples: [2502, 0]

In [12]:
print("rs 라이브러리 : "+ str(r2_score(total, total_result)))
from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(total,total_result)
# MAE_grd_length=mean_absolute_error(total, total_result_grd_length)
# MAE_grd_size=mean_absolute_error(total, total_result_grd_size)
# MAE_grd_length_resolution=mean_absolute_error(total, total_result_grd_length_resolution)
# MAE_grd_size_resolution=mean_absolute_error(total, total_result_grd_size_resolution)
# MAE_new_con=mean_absolute_error(total, total_result_new_con)
print("MAE : "+str(mean_absolute_error(total, total_result)))
# print("MAE - grd_length: "+str(mean_absolute_error(total, total_result_grd_length)))
# print("MAE - grd_size : "+str(mean_absolute_error(total, total_result_grd_size)))
# print("MAE - grd_length_resolution : "+str(mean_absolute_error(total, total_result_grd_length_resolution)))
# print("MAE - grd_size_resolution : "+str(mean_absolute_error(total, total_result_grd_size_resolution)))
# print("MAE - 새로운 상헌 상수 : "+str(mean_absolute_error(total, total_result_new_con)))

from sklearn.metrics import mean_squared_error 
MSE = mean_squared_error(total,total_result)
# MSE_grd_length=mean_squared_error(total, total_result_grd_length)
# MSE_grd_size=mean_squared_error(total, total_result_grd_size)
# MSE_grd_length_resolution=mean_squared_error(total, total_result_grd_length_resolution)
# MSE_grd_size_resolution=mean_squared_error(total, total_result_grd_size_resolution)
# MSE_new_con=mean_squared_error(total, total_result_new_con)
print("MSE : "+str(mean_squared_error(total,total_result )))
# print("MSE - grd_length: "+str(mean_squared_error(total, total_result_grd_length)))
# print("MSE - grd_size : "+str(mean_squared_error(total, total_result_grd_size)))
# print("MSE - grd_length_resolution : "+str(mean_squared_error(total, total_result_grd_length_resolution)))
# print("MSE - grd_size_resolution : "+str(mean_squared_error(total, total_result_grd_size_resolution)))
# print("MSE - 새로운 상헌 상수 : "+str(mean_squared_error(total, total_result_new_con)))

from sklearn.metrics import mean_squared_error 
print("RMSE : "+str(np.sqrt(MSE)))

rs 라이브러리 : 0.9624299332995595
MAE : 0.26088329336530774
MSE : 0.13063593125499598
RMSE : 0.36143592966803395
